## Recuperação da Informação
####Professor: Leandro Balby
####Aluno: Lucas Barros Rocha - 115111579

In [60]:
#bibliotecas

import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
import collections
import time
import re
import heapq

nltk.download('stopwords')

#palavras a serem desconsideradas
tknz = RegexpTokenizer(r'(\b[A-Za-z][a-z]{2,27}\b)')
stopwords = nltk.corpus.stopwords.words('portuguese')

#instalação de biblioteca não nativa no colab 
!pip install memory_profiler


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Importando os Dados, que são: notícias retiradas por crawler+scraper de portais de notícas online.

In [0]:
result = pd.read_csv('https://raw.githubusercontent.com/lucasbarrosrocha/rec-info/master/dataframes/results.csv')

#### 1. Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.


In [0]:
indices = {}
n = 0

for text in result.text:
  palavras = [palavra for palavra in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)
    
for elem in indices.items():
  d = dict(collections.Counter(elem[1]))
  indices[elem[0]] = list(d.items())

In [0]:
tabela_de_tokens = pd.DataFrame()

tabela_de_tokens['Palavra'] = [palavra for palavra in indices.keys()]
tabela_de_tokens['Documentos'] = [docs for docs in indices.values()]

tabela_de_tokens.to_csv('tabela.csv', encoding='utf-8', index=False)

#tabela_de_tokens

#### 2. Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).

In [0]:
def documento_por_vez(consulta, indices, n):
    start = time.time()
    
    indices_consultas = []
    rank = []
    
    for palavra in consulta.split(" "):
      if palavra in indices.keys():
          indices_consultas.append(indices[palavra])
    
    for doc in range(1, len(result.text)+1):
      score_d = 0
      for indice in indices_consultas:
          for i in indice:
              if i[0] == doc:
                  score_d += i[1]
                  break
      if score_d != 0:
        heapq.heappush(rank, (score_d, doc))
    
    end = time.time() - start
    return heapq.nlargest(k, rank), end
  
  
def termo_por_vez(consulta, indices, n):
  start = time.time()

  a = {}
  indices_consultas = []
  rank = []

  for palavra in consulta.split(" "):
    if palavra in indices.keys():
        indices_consultas.append(indices[palavra])

  for indice in indices_consultas:
    for i in indice:
        doc = i[0]
        if doc not in a.keys():
            a[doc] = 0
        a[doc] += i[1]

  for (doc, score) in a.items():
    heapq.heappush(rank, (score, doc))

  end = time.time() - start
  return heapq.nlargest(k, rank), end

In [0]:
consultas = ["brasil", "governo", "futebol", "economia", "cultura"]

In [0]:
resultados_documentos = []
resultados_termos = []
tempos_documentos = []
tempos_termos = []


n = 10

for consulta in consultas:
  score_doc, time_doc = documento_por_vez(consulta, indices, n)
  resultados_documentos.append(score_doc)
  tempos_documentos.append(time_doc)
  
  score_term, time_term = termo_por_vez(consulta, indices, n)
  resultados_termos.append(score_term)
  tempos_termos.append(time_term)

In [67]:
df_consulta = pd.DataFrame()
df_consulta['Consulta'] = consultas
df_consulta['Documento de cada vez'] = resultados_documentos
df_consulta['Termo de cada vez'] = resultados_termos
df_consulta['Comparação'] = df_consulta['Documento de cada vez'] == df_consulta['Termo de cada vez']
df_consulta.index+=1
df_consulta

,Consulta,Documento de cada vez,Termo de cada vez,Comparação
1,brasil,"[(47, 151), (15, 166), (11, 19), (10, 26), (9,...","[(47, 151), (15, 166), (11, 19), (10, 26), (9,...",True
2,governo,"[(15, 173), (13, 166), (10, 248), (10, 115), (...","[(15, 173), (13, 166), (10, 248), (10, 115), (...",True
3,futebol,"[(8, 242), (7, 118), (7, 47), (6, 115), (5, 21...","[(8, 242), (7, 118), (7, 47), (6, 115), (5, 21...",True
4,economia,"[(10, 138), (8, 125), (6, 127), (6, 69), (6, 3...","[(10, 138), (8, 125), (6, 127), (6, 69), (6, 3...",True
5,cultura,"[(10, 124), (4, 158), (3, 248), (3, 165), (2, ...","[(10, 124), (4, 158), (3, 248), (3, 165), (2, ...",True


In [68]:

df_tempo_consulta = pd.DataFrame()
df_tempo_consulta['Tempo médio: documento por vez (ms)'] = tempos_documentos
df_tempo_consulta['Tempo médio: Termo de cada vez (ms)'] = tempos_termos
df_tempo_consulta.index+=1
df_tempo_consulta

,Tempo médio: documento por vez (ms),Tempo médio: Termo de cada vez (ms)
1,0.001426,0.000074
2,0.001405,0.000072
3,0.000401,0.000017
4,0.000776,0.000161
5,0.000612,0.000018


In [69]:
#declarando medidor de memória
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


##### Uso da meória no Documento


In [70]:
%memit documento_por_vez(consultas[0], indices, n)

peak memory: 212.62 MiB, increment: 0.00 MiB


##### Uso da memória no Termo

In [71]:
%memit termo_por_vez(consultas[0], indices, n)

peak memory: 212.62 MiB, increment: 0.00 MiB


#### 3. Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

In [0]:
def consulta_conjuntiva(consulta, indices, n):
  
  indices_consultas = [] # Array
  rank = [] # HeapQueue, implementation of PriorityQueue
  
  for palavra in consulta.split(" "):
    if palavra in indices.keys():
        indices_consultas.append(indices[palavra])
  
  todos_indices = [item for sublist in indices_consultas for item in sublist]
  todos_indices.sort() # Sorted Array of all indices from consulta terms
  
  for i in range(len(todos_indices)):
    score_d = 0
    d = todos_indices.pop()
    r = 1
    for indice in todos_indices:
      if indice[0] == d[0]:
        score_d += indice[1]
        r += 1
    if score_d != 0 and r == len(indices_consultas):
      score_d += d[1]
      heapq.heappush(rank, (score_d, d[0]))
        
  return heapq.nlargest(n, rank)

In [0]:
consultas_conjunta = ["novos ajustes", "futebol brasileiro", "tudo sobre", "estados unidos", "jair bolsonaro"]

In [76]:

n = 10
resultados_conjuntos = []

for consulta in consultas_conjunta:
  score_conjuntos = consulta_conjuntiva(consulta, indices, n)
  resultados_conjuntos.append(score_conjuntos)

df_consultas_conjutiva = pd.DataFrame()
df_consultas_conjutiva['Consulta'] = consultas_conjunta
df_consultas_conjutiva['Resultados'] = resultados_conjuntos
df_consultas_conjutiva.index+=1
df_consultas_conjutiva

,Consulta,Resultados
1,novos ajustes,"[(2, 121), (2, 107)]"
2,futebol brasileiro,"[(10, 115), (7, 55), (5, 50), (3, 121), (3, 76..."
3,tudo sobre,"[(17, 151), (13, 6), (12, 211), (12, 69), (11,..."
4,estados unidos,"[(8, 69), (8, 66), (8, 34), (6, 173), (6, 28),..."
5,jair bolsonaro,"[(52, 151), (48, 207), (39, 166), (13, 216), (..."
